# Example notebook using Qililab

In [ ]:
import os
from pathlib import Path

import numpy as np
from qibo.gates import CZ, M
from qibo.models.circuit import Circuit

import qililab as ql

%matplotlib widget

In [ ]:
fname = os.path.abspath("")
os.environ["RUNCARDS"] = str(Path(fname) / "runcards")
os.environ["DATA"] = str(Path(fname) / "data")

## Load a platform

In [ ]:
runcard_path = str(Path(fname) / "runcards/galadriel.yml")
platform = ql.build_platform(runcard=runcard_path)

### Platform Chip

In [ ]:
print(platform.chip)

### Platform Buses

In [ ]:
print(platform.buses)

## Connect to a Platform

In [ ]:
# Connect to all instruments of the platform and block the connection for other users
platform.connect(manual_override=False)  # if manual_override=True, it surpasses any blocked connection
platform.initial_setup()  # Sets all the values of the Runcard to the connected instruments
platform.turn_on_instruments()  # Turns on all instruments

## Create an experiment

### Create the circuit associated with the experiment

In [ ]:
# Define Circuit to execute
circuit = Circuit(5)
circuit.add(ql.Drag(q=0, theta=1.23, phase=4.56))
circuit.add(ql.Drag(q=2, theta=2, phase=3))
circuit.add(CZ(2, 3))
circuit.add(ql.Drag(3, 6, 7))
circuit.add(M(0))

### Create loops over parameter values (optional)

In [ ]:
lo_freq_loop = ql.Loop(
    alias="drive_line_bus",
    parameter=ql.Parameter.LO_FREQUENCY,
    values=np.linspace(start=6.0e09, stop=6.5e09, num=10),
)

### Initialize the Experiment

In [ ]:
settings = ql.ExperimentSettings(
    hardware_average=1000,
    repetition_duration=200_000,
    software_average=1,
)

options = ql.ExperimentOptions(
    loops=[lo_freq_loop],  # loops to run the experiment
    settings=settings,  # experiment settings
    name="experiment_name",  # name of the experiment (it will be also used for the results folder name)
)

sample_experiment = ql.Experiment(
    platform=platform,  # platform to run the experiment
    circuits=[circuit],  # circuits to run the experiment
    options=options,  # experiment options
)

### Translate the circuit into pulses and build the Execution object

In [ ]:
sample_experiment.build_execution()

### Draw pulses to be sent

In [ ]:
sample_experiment.pulse_schedules[0].elements

In [ ]:
figure = sample_experiment.draw()

## Check the compiled sequences of the experiment

In [ ]:
# this method compiles the pulses into the assembly program and returns all the sequences sent to the instruments
sequences = sample_experiment.compile()

Program of the drive line bus:

In [ ]:
sequences[0]["drive_line_q0_bus"][0]._program

Program of the feedline input output bus:

In [ ]:
sequences[0]["feedline_bus"][0]._program

## Run the experiment

In [ ]:
results = sample_experiment.run()

In [ ]:
print(results)

## Disconnect from a Platform

- Disconnect all instruments specified in the loaded platform
- Turns off the instruments only when explicitly specified (by default is set to False)

In [ ]:
platform.disconnect()